# RuBisCO annotations

This notebook contains the code that was used to annotate the set of 72K RuBisCOs collected for the RuBisCOlympics project.

## A. Preparing the data for the tree

### 1. Read the usearch clustering

In [121]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

sequence_info = pd.read_csv("data/RuBisCO.300-700.faa.sorted.0.9.uc", sep="\t", header=None).iloc[:, [0, 1, 2, 3, 8]]
sequence_info.columns = ["membership", "cluster", "length", "pidentity", "sequence"]
cluster_info = pd.DataFrame(sequence_info.loc[sequence_info.membership=="C", :])
cluster_info.set_index(["cluster"], inplace=True)
cluster_info.drop(["membership", "pidentity"], axis=1, inplace=True)
cluster_info.columns = ["size", "seed"]

sequence_info = pd.DataFrame(sequence_info.loc[sequence_info.membership!="C", :])
sequence_info.set_index(["sequence"], inplace=True)
#print(sequence_info.loc["OPY85882.1", :])
print(f"A total of {sequence_info.shape[0]} sequences were clustered into {cluster_info.shape[0]} clusters")

A total of 72395 sequences were clustered into 4236 clusters


### 2. Add the type information to the sequences

In [122]:
sequence_info["type"] = "Unknown"

lists = {"I-Eukaryotes": "data/Type_I.Eukaryota.list", "II": "data/Type_II.list", "II-III": "data/Type_II_III.list", "III": "data/Type_III.list"}
for form, list_file in lists.items():
    with open(list_file) as fin:
        ids = [x.strip() for x in fin.readlines()]
        while "" in ids:
            ids.remove("")
        print(f"{len(ids)} {form} RuBisCOs")
        sequence_info.loc[ids, "type"] = form

table_files = ["data/Type_I.non-Eukaryota.info.txt", "data/Type_IV.info.txt"]
for file in table_files:
    table = pd.read_csv(file, sep="\t", header=None)
    ids = list(table[0])
    forms = [str(x).replace("Type_1", "I-") for x in table[2]]
    forms = [str(x).replace("Type_I/", "I-") for x in forms]
    forms = [str(x).replace("I-\'\'", "I-New") for x in forms]
    forms = [str(x).replace("I-\'", "I-Banda") for x in forms]
    forms = [str(x).replace("nan", "Unknown") for x in forms]
    print(f"{len(ids)} RuBisCOs from {file}")
    sequence_info.loc[ids, "type"] = forms

print(sequence_info["type"].value_counts())
print(sum(sequence_info["type"].value_counts()))
#sequence_info.to_csv("kuku.csv", sep="\t")

51859 I-Eukaryotes RuBisCOs
66 II RuBisCOs
24 II-III RuBisCOs
364 III RuBisCOs
4302 RuBisCOs from data/Type_I.non-Eukaryota.info.txt
4680 RuBisCOs from data/Type_IV.info.txt
I-Eukaryotes      51859
Unknown           11121
IV-Non-photo       2692
I-CD               1727
I-A                1302
IV-DeepYkr          749
I-B                 705
III                 364
IV-GOS_Global       339
I-E                 337
IV-Ykr-2            294
IV-Photo            229
IV-NG1              117
IV-NG2              113
I-Banda              69
IV-Ykr               66
II                   65
I-G                  61
IV-NG3               61
I-Unclassified       59
II-III               24
I-F                  21
I-New                21
Name: type, dtype: int64
72395


### 3. Add the type information to the clusters

In [123]:
types = []
types_info = []
for c in cluster_info.index:
    cluster_seqs = sequence_info[sequence_info["cluster"]==c]
    t = cluster_seqs["type"].value_counts()
    if len(t) == 1:
        types.append(t.index[0])
        types_info.append("")
    elif len(t) == 2 and "Unknown" in t.index:
        this_type = t.index[0] if t.index[0] != "Unknown" else t.index[1]
        types.append(f"{this_type}/Unknown")
        types_info.append(f"{t.index[0]}: {t[0]}, {t.index[1]}: {t[1]}")
    else:
        types.append("Ambiguous")
        info = ""
        for i in range(len(t)):
            info += f", {t.index[i]}: {t[i]}"
        info = info[2:]
        types_info.append(info)
    if len(types)%500 == 0:
        print(len(types))
cluster_info["type"] = types
cluster_info["type_info"] = types_info
print(cluster_info["type"].value_counts())

500
1000
1500
2000
2500
3000
3500
4000
Unknown                 1100
IV-Non-photo             833
IV-DeepYkr               489
III                      240
IV-Ykr-2                 212
I-Eukaryotes/Unknown     197
I-CD                     194
IV-GOS_Global            123
IV-Photo                 119
I-Eukaryotes             116
I-A                      108
IV-NG1                    67
IV-NG2                    67
I-E                       55
IV-Ykr                    36
Ambiguous                 34
I-Unclassified            29
I-Banda                   27
III/Unknown               26
IV-NG3                    23
I-G                       22
I-B                       18
I-New                     15
I-F                       14
II/Unknown                13
I-CD/Unknown              11
II-III                    10
IV-Non-photo/Unknown      10
I-A/Unknown                9
II-III/Unknown             5
I-B/Unknown                4
II                         3
IV-Ykr/Unknown             2
I-Ba

### 4. Create the fasta file for the cluster representatives

In [124]:
import os

os.mkdir("for-tree")
from Bio.SeqRecord import SeqRecord
reps = []
type2count = {}
cluster_info["tree_name"] = "Unnamed"
for record in SeqIO.parse("data/RuBisCO.300-700.faa", 'fasta'):
    if sequence_info.loc[record.id, "membership"] != "S":
        continue
    cluster = sequence_info.loc[record.id, "cluster"]
    cluster_type = cluster_info.loc[cluster, "type"]
    type2count[cluster_type] = type2count.get(cluster_type, 0) + 1
    new_record = SeqRecord(id = f"{cluster_type}.{type2count[cluster_type]}", seq = record.seq, description = "")
    reps.append(new_record)
    cluster_info.loc[cluster, "tree_name"] = f"{cluster_type}.{type2count[cluster_type]}"

SeqIO.write(reps, "for-tree/RuBisCO.300-700.0.9.faa", "fasta")
cluster_info.to_csv("for-tree/RuBisCO.300-700.0.9.faa.csv")

### 5. Add annotations to sequences in X/Unknown clusters

In [127]:
print("*** Before ***")
print(sequence_info["type"].value_counts())
for cluster in cluster_info.index:
    if cluster_info.loc[cluster, "type"].endswith("/Unknown"):
        seq_type = cluster_info.loc[cluster, "type"][:-len("/Unknown")]
        sequence_info.loc[sequence_info["cluster"]==cluster, "type"] = seq_type

print("\n*** After ***")
print(sequence_info["type"].value_counts())
#sequence_info.to_csv("RuBisCO.300-700.faa.csv")

*** Before ***
I-Eukaryotes      51859
Unknown           11121
IV-Non-photo       2692
I-CD               1727
I-A                1302
IV-DeepYkr          749
I-B                 705
III                 364
IV-GOS_Global       339
I-E                 337
IV-Ykr-2            294
IV-Photo            229
IV-NG1              117
IV-NG2              113
I-Banda              69
IV-Ykr               66
II                   65
I-G                  61
IV-NG3               61
I-Unclassified       59
II-III               24
I-F                  21
I-New                21
Name: type, dtype: int64

*** After ***
I-Eukaryotes      55888
Unknown            6788
IV-Non-photo       2703
I-CD               1741
I-A                1314
IV-DeepYkr          750
I-B                 710
III                 424
IV-GOS_Global       339
I-E                 337
IV-Ykr-2            295
II                  240
IV-Photo            229
IV-NG1              117
IV-NG2              113
I-Banda              71
IV-Ykr   

## B. Creating the tree

The tree was created on a Linux server using the following commands:

```
#!/bin/bash

trimal_threshold=0.05
muscle_maxiters=2
filename=RuBisCO.300-700.0.9.faa
infile=RuBisCO.300-700.0.9.faa
alignment_file=$filename.muscle_maxiters_$muscle_maxiters.aln
trimal_file=$alignment_file.trimal_$trimal_threshold.aln
fasttree_file=$trimal_file.fasttree.nwk

date > $alignment_file.stderr
muscle -in $infile -out $alignment_file -maxiters $muscle_maxiters -log $alignment_file.stdout 2>> $alignment_file.stderr
date >> $alignment_file.stderr
trimal -in $alignment_file -out temp.aln -gt 0.05
sed 's/ .*//' temp.aln > $trimal_file

fasttree < $trimal_file > $fasttree_file 2> $fasttree_file.stderr
```

## C. Annotating the tree and getting the final annotations for the sequences

The tree was manually annotated and the leaves for the different groups were kept in their respective groups. Next we update the cluster_info and sequence_info dataframes, and save the final result

In [139]:
import os
import glob

os.mkdir("final")
cluster_info.rename(columns = {'type':'annotation_preliminary'}, inplace = True)
cluster_info['annotation'] = "Unknown"
for list_file in glob.glob("tree/*.list"):
    cluster_type = list_file[len("tree/"):-len(".list")]
    with open(list_file) as fin:
        for tree_name in fin:
            tree_name = tree_name.strip().replace(" ", "_")
            cluster_info.loc[(cluster_info["tree_name"]==tree_name), 'annotation'] = cluster_type

print(cluster_info["annotation"].value_counts())
cluster_info.to_csv("final/RuBisCO.300-700.0.9.faa.csv")
cluster_info

IV-NonPhoto    843
III            703
IV-Deep-Ykr    492
I-CD2          489
I-CD1          284
II             257
IV-Ykr-2       211
IV-GOS         127
I-A            127
IV-Photo       121
IV-NG3         119
Unknown        116
IV-NG2          91
IV-NG1          67
II-III          60
IV-Ykr          37
I-Banda         29
IV-Unknown      29
I-B             20
I-NG1           14
Name: annotation, dtype: int64


,size,seed,annotation_preliminary,type_info,tree_name,annotation
cluster,,,,,,
0,38,AAQ04822.2,II/Unknown,"II: 37, Unknown: 1",II/Unknown.11,II
1,54,WP_106857372.1,I-CD,,I-CD.158,I-CD2
2,115,RWP03589.1,I-CD,,I-CD.125,I-CD2
3,16578,QHN53519.1,Ambiguous,"I-Eukaryotes: 14300, Unknown: 2268, I-B: 10",Ambiguous.1,I-CD1
4,1724,BBE28301.1,Ambiguous,"I-Eukaryotes: 1600, Unknown: 115, I-CD: 9",Ambiguous.16,I-CD2
...,...,...,...,...,...,...
4231,1,TARA_125.SAMEA2622837.140.0.22-3_137106_7,Unknown,,Unknown.1041,IV-Unknown
4232,1,TARA_038.SAMEA2620014.5.0-0.22_8457_6,IV-GOS_Global,,IV-GOS_Global.78,IV-GOS
4233,1,NGP75567.1,IV-DeepYkr,,IV-DeepYkr.193,IV-Deep-Ykr


In [140]:
sequence_info.rename(columns = {'type':'annotation_preliminary'}, inplace = True)
sequence_info['annotation'] = "Unknown"
for cluster in cluster_info.index:
    sequence_info.loc[sequence_info["cluster"]==cluster, "annotation"] = cluster_info.loc[cluster, "annotation"]

print(sequence_info["annotation"].value_counts())
sequence_info.to_csv("final/RuBisCO.300-700.faa.csv")
sequence_info

I-CD1          48551
I-CD2          14499
IV-NonPhoto     2703
I-A             1332
III             1249
II               855
IV-Deep-Ykr      752
I-B              584
IV-GOS           343
IV-Ykr-2         291
IV-Photo         231
IV-NG3           193
IV-NG2           175
Unknown          175
II-III           148
IV-NG1           117
I-Banda           71
IV-Ykr            67
IV-Unknown        39
I-NG1             20
Name: annotation, dtype: int64


,membership,cluster,length,pidentity,annotation_preliminary,annotation
sequence,,,,,,
AAQ04822.2,S,0,538,*,II,II
WP_106857372.1,S,1,497,*,I-CD,I-CD2
RWP03589.1,S,2,497,*,I-CD,I-CD2
QHN53519.1,S,3,495,*,I-Eukaryotes,I-CD1
YP_009729450.1,H,3,492,93.6,I-Eukaryotes,I-CD1
...,...,...,...,...,...,...
TARA_124.SAMEA2622801.120.0.45-0.8_267714_17,H,775,305,98.0,IV-GOS_Global,IV-GOS
ABP82249.1,H,25,304,99.3,I-Eukaryotes,I-CD1
CDI73530.1,H,58,303,96.7,I-Eukaryotes,I-CD2


In [141]:
records = []
for record in SeqIO.parse("data/RuBisCO.300-700.faa", 'fasta'):
    form=sequence_info.loc[record.id, "annotation"]
    new_record = SeqRecord(id=record.id, description=f"/RuBisCO_Form={form} {record.description}", seq=record.seq)
    records.append(new_record)
    
SeqIO.write(records, "final/RuBisCO.300-700.faa", "fasta")

72395